Importamos Librerías que vamos a utilizar

In [99]:
# Tratamiento de datos
import numpy as np
import pandas as pd

# Gráficos
import matplotlib.pyplot as plt

# Preprocesado y modelado
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn import preprocessing
from sklearn import metrics

Ingesta de datos: en este caso contamos con 2 archivos .csv que contienen información sobre la estadía de pacientes en instituciones hospitalarias. Se pretende generar un modelo de clasificación que permita predecir si un paciente tendrá una estadía larga (más de 8 días) o corta (8 días o menos) en función de las variables disponibles.

In [100]:
train ='./datasets/hospitalizaciones_train.csv'
df_train= pd.read_csv(train)

test ='./datasets/hospitalizaciones_test.csv'
df_test= pd.read_csv(test)



Empezamos con nuestro Análisis exploratorio de datos!

ACLARACIÓN: si bien vamos a utilizar df_train para entrenar nuestro modelo y df_test para buscar las predicciones, los cambios aplicados como parte del EDA se realizarán en ambos dataframes.

Utilizo .head() .info() .duplicate() para obtener un panoráma general de mi dataframe, ver los tipos de variables, si hay filas duplicadas y si existen valores nulos.

In [101]:
df_train.head()

,Available Extra Rooms in Hospital,Department,Ward_Facility_Code,doctor_name,staff_available,patientid,Age,gender,Type of Admission,Severity of Illness,health_conditions,Visitors with Patient,Insurance,Admission_Deposit,Stay (in days)
0,4,gynecology,D,Dr Sophia,0,33070,41-50,Female,Trauma,Extreme,Diabetes,4,Yes,2966.408696,8
1,4,gynecology,B,Dr Sophia,2,34808,31-40,Female,Trauma,Minor,Heart disease,2,No,3554.835677,9
2,2,gynecology,B,Dr Sophia,8,44577,21-30,Female,Trauma,Extreme,Diabetes,2,Yes,5624.733654,7
3,4,gynecology,D,Dr Olivia,7,3695,31-40,Female,Urgent,Moderate,None,4,No,4814.149231,8
4,2,anesthesia,E,Dr Mark,10,108956,71-80,Male,Trauma,Moderate,Diabetes,2,No,5169.269637,34


In [102]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410000 entries, 0 to 409999
Data columns (total 15 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Available Extra Rooms in Hospital  410000 non-null  int64  
 1   Department                         410000 non-null  object 
 2   Ward_Facility_Code                 410000 non-null  object 
 3   doctor_name                        410000 non-null  object 
 4   staff_available                    410000 non-null  int64  
 5   patientid                          410000 non-null  int64  
 6   Age                                410000 non-null  object 
 7   gender                             410000 non-null  object 
 8   Type of Admission                  410000 non-null  object 
 9   Severity of Illness                410000 non-null  object 
 10  health_conditions                  410000 non-null  object 
 11  Visitors with Patient              4100

In [103]:
df_train.duplicated().sum()

0

Como mi modelo busca responder un problemática de clasificación (estadía larga, estadía corta), lo primero que tenemos que hacer es transformar nuestra variable numérica 'Stay (in days)' en una variable Binaria: 0 (≤ 8 días = estadía corta) y 1 (> 8 días = estadía larga), esta transformación va a estar reflejada en una nueva columna llamada 'Estadia'.

In [104]:
df_train["Estadia"] = np.where(df_train["Stay (in days)"]>8,1,0)


Ya tenemos nuestra nueva columna, ahora pasamos a eliminar la columna 'Stay (in days)', también elimino otras columnas que considero no tienen correlacion con la predicción que queremos hacer.
-Ward_Facility_Code: Código de la habitación del paciente.
-doctor_name: Nombre de el/la doctor/a a cargo del paciente.
-staff_available: Cantidad de personal disponible al momento del ingreso del paciente.
-patientid: Identificador del paciente.
-Visitors with Patient: Cantidad de visitantes registrados para el paciente.

In [105]:
df_train = df_train.drop(columns=['Stay (in days)','Ward_Facility_Code','doctor_name','staff_available','patientid','Visitors with Patient'])

In [106]:
df_test = df_test.drop(columns=['Ward_Facility_Code','doctor_name','staff_available','patientid','Visitors with Patient'])

In [107]:
df_train.head()

,Available Extra Rooms in Hospital,Department,Age,gender,Type of Admission,Severity of Illness,health_conditions,Insurance,Admission_Deposit,Estadia
0,4,gynecology,41-50,Female,Trauma,Extreme,Diabetes,Yes,2966.408696,0
1,4,gynecology,31-40,Female,Trauma,Minor,Heart disease,No,3554.835677,1
2,2,gynecology,21-30,Female,Trauma,Extreme,Diabetes,Yes,5624.733654,0
3,4,gynecology,31-40,Female,Urgent,Moderate,None,No,4814.149231,0
4,2,anesthesia,71-80,Male,Trauma,Moderate,Diabetes,No,5169.269637,1


Para poder obtener nuestras predicciones vamos a utilizar el modelo Random Forest, éste está formado por un conjunto de árboles de decisión individuales, la predicción de una nueva observación se obtiene agregando las predicciones de todos los árboles individuales que forman el modelo.

Ventajas:

-Pueden aplicarse a problemas de regresión y clasificación.

-No se ven muy influenciados por outliers.

-Son muy útiles en la exploración de datos, permiten identificar de forma rápida y eficiente las variables (predictores) más importantes.

-Tienen buena escalabilidad, pueden aplicarse a conjuntos de datos con un elevado número de observaciones.



Para poder comenzar a entrenar el modelo, voy a necesitar transformar las variables categóricas en variables numéricas, para ello voy a utilizar .LabelEncoder().

In [111]:
#Primero para el dataframe df_train

cat = preprocessing.LabelEncoder()

cat.fit(df_train.Department)
df_train.Department = cat.transform(df_train.Department)

cat.fit(df_train.Age)
df_train.Age = cat.transform(df_train.Age)

cat.fit(df_train.gender)
df_train.gender = cat.transform(df_train.gender)

cat.fit(df_train['Type of Admission'])
df_train['Type of Admission'] = cat.transform(df_train['Type of Admission'])

cat.fit(df_train['Severity of Illness'])
df_train['Severity of Illness'] = cat.transform(df_train['Severity of Illness'])

cat.fit(df_train.health_conditions)
df_train.health_conditions = cat.transform(df_train.health_conditions)

cat.fit(df_train.Insurance)
df_train.Insurance = cat.transform(df_train.Insurance)

In [112]:
#Después para df_test

cat = preprocessing.LabelEncoder()

cat.fit(df_test.Department)
df_test.Department = cat.transform(df_test.Department)

cat.fit(df_test.Age)
df_test.Age = cat.transform(df_test.Age)

cat.fit(df_test.gender)
df_test.gender = cat.transform(df_test.gender)

cat.fit(df_test['Type of Admission'])
df_test['Type of Admission'] = cat.transform(df_test['Type of Admission'])

cat.fit(df_test['Severity of Illness'])
df_test['Severity of Illness'] = cat.transform(df_test['Severity of Illness'])

cat.fit(df_test.health_conditions)
df_test.health_conditions = cat.transform(df_test.health_conditions)

cat.fit(df_test.Insurance)
df_test.Insurance = cat.transform(df_test.Insurance)

In [113]:
#Observamos los cambios luego de finalizar la normalización de datos

df_train.head()

,Available Extra Rooms in Hospital,Department,Age,gender,Type of Admission,Severity of Illness,health_conditions,Insurance,Admission_Deposit,Estadia
0,4,2,4,0,1,0,1,1,2966.408696,0
1,4,2,3,0,1,1,2,0,3554.835677,1
2,2,2,2,0,1,0,1,1,5624.733654,0
3,4,2,3,0,2,2,4,0,4814.149231,0
4,2,1,7,1,1,2,1,0,5169.269637,1


In [90]:
#Establecemos nuestras variables x e y

x= df_train.drop(['Estadia'], axis= 1)
y= df_train['Estadia']

In [91]:
# División de los datos en train y test

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

Entrenamiento de Random Forest:
El algoritmo de Random Forest es una modificación del proceso de bagging que consigue mejorar los resultados gracias a que decorrelaciona aún más los árboles generados en el proceso, consiguiendo reducir la varianza.


Si bien RandomForestClassifier() tiene valores por defecto para sus hiperparámetros, no se puede saber si estos son los más adecuados, la forma de identificarlos es mediante el uso de estrategias de validación, por ejemplo validación cruzada (aunque esto último puede ser computacionalmente costoso).

In [114]:
model = RandomForestClassifier()

In [115]:
#Entrenamos el modelo con los datos de entrenamiento

model.fit(X_train, y_train)

RandomForestClassifier()

In [117]:
#Hacemos las predicciones con los datos de testo (nuevos)

prediction = model.predict(X_test)


In [118]:
#Analizamos la matriz de confusión
mat_c = confusion_matrix(y_test,prediction)
print("Matriz de confusión:")
print(mat_c)

Matriz de confusión:
[[25192 13622]
 [13198 50488]]


In [119]:
#Analizamos el reporte de clasificación (podemos observar el recall y accuracy)

report = classification_report(y_test, prediction)
print("Reporte de Clasificación:")
print(report)

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.66      0.65      0.65     38814
           1       0.79      0.79      0.79     63686

    accuracy                           0.74    102500
   macro avg       0.72      0.72      0.72    102500
weighted avg       0.74      0.74      0.74    102500



In [120]:
#Imprimimos las métricas más importantes que evaluan nuestro modelo

print("accuracy: ", metrics.accuracy_score(y_test,prediction))
print("Recall: ", metrics.recall_score(y_test,prediction))

accuracy:  0.7383414634146341
Recall:  0.7927645008322081


Ahora si podemos pasar a predecir con nuetsro modelo la estadía de los pacientes, vamos a utilizar df_test y la columna con los resultados se va a llamar 'pred'.

In [121]:
#Repetimos los mismos pasos
#Entrenamos...

model_pred = RandomForestClassifier()
model_pred.fit(x,y)

RandomForestClassifier()

In [122]:
#Para luego obtener nuestra columna 'pred' con las predicciones

prediction1 = pd.DataFrame()
prediction1['pred'] = model_pred.predict(df_test)

In [123]:
#Como último paso guardamos la columna en un archivo .csv para que pueda ser evaluado nuestro modelo.

prediction1.to_csv(path_or_buf='../MelinaRG.csv' , index=False)